# Migration tool "analitics" notebooks

Generate legacy list of tools:
```bash 
nomad dev  toolkit-metadata > tutorials.jso
```

In [ ]:
import json

with open('tutorials.json') as f:
    tutorials = json.load(f)['tutorials']

print(tutorials)

In [ ]:
keys = set()
for tutorial in tutorials:
    keys.update(list(tutorial.keys()))
keys

In [ ]:
keys = set()
for tutorial in tutorials:
    keys.update(list(tutorial.get('flags', {}).keys()))
'flags', keys

In [ ]:
keys = set()
for tutorial in tutorials:
    keys.update(list(tutorial['labels'].keys()))
keys

In [ ]:
keys = set()
for tutorial in tutorials:
    keys.update(list(tutorial.get('labels').get('application_keyword', [])))
keys

In [ ]:
keys = set()
for tutorial in tutorials:
    keys.update(list(tutorial.get('labels').get('application_section', [])))
keys

In [ ]:
keys = set()
for tutorial in tutorials:
    keys.update(list(tutorial.get('labels').get('application_system', [])))
keys

In [ ]:
keys = set()
for tutorial in tutorials:
    keys.update(list(tutorial.get('labels').get('category')))
keys

In [ ]:
keys = set()
for tutorial in tutorials:
    keys.update(list(tutorial.get('labels').get('ai_methods', [])))
keys

In [ ]:
keys = set()
for tutorial in tutorials:
    keys.update(list(tutorial.get('labels').get('language', [])))
keys

In [ ]:
keys = set()
for tutorial in tutorials:
    keys.update(list(tutorial.get('labels').get('platform')))
keys

In [ ]:
tutorial = tutorials[0]
tutorial

In [ ]:
authors = []
for author in tutorial.get('authors'):
    last_name, first_name = author.split(',', 1)
    authors.append({'last_name': last_name.strip(), 'first_name': first_name.strip()})

authors[-1]['email'] = tutorial.get('email')
authors

In [ ]:
map_categories = {
    'advanced_tutorial': 'Advanced tutorial',
    'beginner_tutorial': 'Beginner tutorial',
    'intermediate_tutorial': 'Intermediate tutorial',
    'query_tutorial': 'Query tutorial',
}

In [ ]:
def build_new_tutorial(tutorial):
    new = {
        'm_def': 'nomad_aitoolkit.schema.AIToolkitNotebook',
        'name': tutorial['title'],
        'description': tutorial['description'],
        'date': tutorial['updated'],
        'category': map_categories[tutorial['labels']['category'][0]],
        'methods': [{'name': v} for v in tutorial['labels']['ai_methods']],
        'applications': [{'name': v} for v in tutorial['labels']['application_system']],
        'platform': 'Python',
    }

    new['authors'] = []
    for author in tutorial['authors']:
        last_name, first_name = author.split(',', 1)

        new['authors'].append(
            {'last_name': last_name.strip(), 'first_name': first_name.strip()}
        )

    # print(tutorial.get('email'))
    # new['authors'][-1]['email'] = tutorial.get('email')

    new['references'] = []

    if tutorial.get('link_doi_paper'):
        # new['related_publications'] = [
        #     {
        #         'DOI_number': tutorial.get('link_doi_paper')
        #     }
        # ]

        new['references'].append(
            {'kind': 'article_doi', 'uri': tutorial.get('link_doi_paper')}
        )

    if tutorial.get('link_public'):
        new['references'].append({'kind': 'hub', 'uri': tutorial['link_public']})

    if tutorial.get('link_paper'):
        new['references'].append({'kind': 'article_url', 'uri': tutorial['link_paper']})

    if tutorial.get('url'):
        new['references'].append({'kind': 'repository', 'uri': tutorial['url']})

    if tutorial.get('link_video'):
        new['references'].append({'kind': 'video', 'uri': tutorial['link_video']})

    slug = tutorial['url'].rsplit('/', 1)[1]
    if slug.startswith('analytics-'):
        slug = slug[10:]

    return slug, {'data': new}


for tutorial in tutorials:
    slug, new_tutorial = build_new_tutorial(tutorial)

    with open(slug + '.archive.json', 'w') as outfile:
        json.dump(new_tutorial, outfile, indent=2)